In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
import glob
import random
import matplotlib.pyplot as plt
from functions import *
from utils import *

Data

In [ ]:
# Bangla sign data
## Kaggle data
digit_files = glob.glob("./digit/*/*.jpg")
## Custom data
custom_files = glob.glob("./data_custom/*/*.jpeg")

In [ ]:
# data
random_img = random.choice(custom_files + digit_files)
orig_img = cv2.imread(random_img)

# skin-color-detection
hand_sign, skin_mask, largest_contour, x, y = find_skin_color_blobs(orig_img)
skin_mask = preprocess(skin_mask)
hist = cv2.calcHist([hand_sign], [0], None, [256], [0, 256])

# projection-profile
bounded_skin_mask, horiz, vert = find_projection_bounding_box(hand_sign)


# Plot - Grayscaling, Histogram
plt.figure(figsize=(18, 8))
plt.subplot(2, 3, 1)
plt.imshow(hand_sign)
plt.subplot(2, 3, 2)
plt.imshow(skin_mask, cmap="gray")
# plt.axis("off")
plt.subplot(2, 3, 3)
plt.plot(hist)
plt.xlim(0, 256)

# Plot - Projection Profile from skin color detected mask
plt.figure(figsize=(18, 8))
plt.subplot(2, 3, 1)
plt.imshow(bounded_skin_mask, cmap="gray")
plt.subplot(2, 3, 2)
plt.plot(np.arange(0, vert.shape[0]), vert)
plt.title("Vertical Projection")
plt.subplot(2, 3, 3)
plt.plot(np.arange(0, horiz.shape[0]), horiz)
plt.title("Horizontal Projection")

In [ ]:
p = get_pyramid(skin_mask)
plt.figure(figsize=(12, 8))
for i in range(len(p)):
    plt.subplot(2, 3, i+1)
    plt.imshow(p[i], cmap="gray")
    plt.title(f"Level: {i+1}")
    plt.axis("off")

In [ ]:
# Access all PNG files in directory
imlist=[cv2.imread(filename) for filename in glob.glob("./data_custom/0/*.jpeg")]
_, skin_mask_template, _, _, _ = find_skin_color_blobs(imlist[0])
skin_mask_template = preprocess(skin_mask_template)
downsampled_skin_mask_template = get_pyramid(skin_mask_template)
plt.imshow(downsampled_skin_mask_template[-1], cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
p_orig = get_pyramid(skin_mask)
method = 'cv2.TM_CCOEFF_NORMED'

for i, p in enumerate(p_orig[::-1]):
    template = downsampled_skin_mask_template[len(downsampled_skin_mask_template) - 1 - i]
    w, h = template.shape[::-1]
    img = p.copy()

    # Template Matching
    res = cv2.matchTemplate(p, template, eval(method))
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    cv2.rectangle(img,top_left, bottom_right, 255, 0)
    plt.subplot(131),plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    plt.subplot(132),plt.imshow(template,cmap = 'gray')
    plt.title('Matching Template'), plt.xticks([]), plt.yticks([])
    plt.subplot(133),plt.imshow(img,cmap = 'gray')
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    plt.show()

In [ ]:
plt.imshow(p[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]], cmap="gray")

Main

In [ ]:
cap = cv2.VideoCapture(0)
ret, prev_frame = cap.read()
if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    exit(0)

cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
cv2.imshow('frame', prev_frame)

cv2.namedWindow('diff', cv2.WINDOW_NORMAL)
cv2.namedWindow('skin', cv2.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    frame_immutable = frame
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    cv2.imshow('frame', frame)

    diff = find_frame_difference(frame, prev_frame)
    cv2.imshow('diff', diff)

    # frame_skin = find_skin_color_blobs(frame)
    frame_skin, skin_mask, largest_contour, bbx_x, bbx_y  = find_skin_color_blobs(frame)
    cv2.imshow('skin', skin_mask)

    frame_shape,_, _, _, _, _ = get_shape_features(frame_skin, largest_contour, bbx_x, bbx_y)
    cv2.imshow('skin', frame_shape)

    prev_frame = frame
    
    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user") 
        break


cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)